# 1. Pytest futtatása terminálból

Eredmények a test/results.json-ben

# 2. Teszteredmény vs elvárt eredmény

Teszt eredmények > results.json (listában van a path)
Elvárt eredmények > graph.json (stringben van az expected path)

### Összehasonlítás

In [60]:
import json
from pathlib import Path
import pandas as pd

current_dir = Path().resolve()

graphs_path = current_dir / "graphs.json"
results_path = current_dir.parent / "results.json"

print("Graphs path:", graphs_path)
print("Results path:", results_path)

with graphs_path.open("r", encoding="utf-8") as f:
    graphs_data = json.load(f)

with results_path.open("r", encoding="utf-8") as f:
    results_data = json.load(f)

print(f"Loaded {len(graphs_data)} graphs.")
print(f"Loaded {len(results_data)} result entries.")

comparison_rows = []

for graph in graphs_data:
    graph_id = graph["graphId"]
    expected = graph["expectedPaths"]

    # tényleges eredmények a results.json-ból
    actual_paths = results_data.get(graph_id)
    
    if not actual_paths:
        # nincs eredmény ehhez a gráfhoz -- valszeg futtani kell a pytestet
        for algo, expected_path in expected.items():
            comparison_rows.append({
                "Graph": graph_id,
                "Algorithm": algo,
                "Expected": expected_path,
                "Actual": "❌ nincs eredmény",
                "Match": "❌"
            })
        continue

    # minden algoritmus összehasonlítása
    for algo, expected_path in expected.items():
        expected_nodes = list(expected_path)  # string(a graph.json-ben) -> lista mint a resoults.json-ben
        actual_path = actual_paths.get(algo, "❌ nincs eredmény")

        if actual_path != "❌ nincs eredmény":
            match = expected_nodes == actual_path
        else:
            match = False

        comparison_rows.append({
            "Graph": graph_id,
            "Algorithm": algo,
            "Expected": expected_path,
            "Actual": actual_path,
            "Match": "✅" if match else "❌"
        })

#táblázat megjeleítése
df = pd.DataFrame(comparison_rows)
df = df.sort_values(by=["Graph", "Algorithm"])
df.reset_index(drop=True, inplace=True)

df.style.applymap(
    lambda v: "color: green" if v == "✅" else ("color: red" if v == "❌" else None),
    subset=["Match"]
)


Graphs path: C:\Users\dojcs\Documents\beadando_mesterseges_intelligenciak_2025_1_python-dinolany-main\tests\resources\graphs.json
Results path: C:\Users\dojcs\Documents\beadando_mesterseges_intelligenciak_2025_1_python-dinolany-main\tests\results.json
Loaded 3 graphs.
Loaded 3 result entries.


C:\Users\dojcs\AppData\Local\Temp\ipykernel_2508\1536998400.py:66: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df.style.applymap(


,Graph,Algorithm,Expected,Actual,Match
0,GRAPH_1,A*,acd,"['a', 'c', 'd']",✅
1,GRAPH_1,beam,abd,"['a', 'b', 'd']",✅
2,GRAPH_1,best-first,abcd,"['a', 'b', 'c', 'd']",✅
3,GRAPH_1,bfs,abd,"['a', 'b', 'd']",✅
4,GRAPH_1,branch-and-bound,acd,"['a', 'c', 'd']",✅
5,GRAPH_1,branch-and-bound-extended-set,acd,"['a', 'c', 'd']",✅
6,GRAPH_1,branch-and-bound-heuristic,acd,"['a', 'c', 'd']",✅
7,GRAPH_1,dfs,abcd,"['a', 'b', 'c', 'd']",✅
8,GRAPH_1,hill-climbing,abcd,"['a', 'b', 'c', 'd']",✅
9,GRAPH_2,A*,SBCEG,"['S', 'B', 'C', 'E', 'G']",✅
